In [1]:
import json
import collections
import numpy as np

In [2]:
with open('../json/formulas.json', 'r') as f:
    formulas = json.load(f)
with open('../json/constants.json', 'r') as f:
    constants = json.load(f)
with open('../json/input_variables.json', 'r') as f:
    input_variables = json.load(f)

# Get dependencies and find a good computation order

## Get dependecies

In [3]:
def get_children(node):
    nodetype = node['nodetype']

    if nodetype == 'symbol':
        name = node['name']
        return set([name])
        
    elif nodetype == 'float':
        return set()
    
    elif nodetype == 'call':
        args = node['args']
        children = set()
        for arg in args:
            children = children | get_children(arg)

        return children
        
    raise ValueError('Unknown type : %s'%nodetype)

In [4]:
children_dict = {}
for name, formula in formulas.items():
    children_dict[name] = get_children(formula)

## Find dependencies without formula

In [5]:
inputs_list = []
for e in input_variables:
    inputs_list.append(e['alias'])
    inputs_list.append(e['name'])

In [6]:
unknown_variables = set()
for k, v in children_dict.items():
    for var in v:
        if (var not in formulas) and (var not in constants) and (var not in inputs_list):
            unknown_variables.add(var)

In [7]:
len(unknown_variables)

1435

## Parents

In [8]:
parents_dict = {}

for k in children_dict:
    parents_dict[k] = set()

for parent, children in children_dict.items():
    for child in children:
        if child in children_dict:
            parents_dict[child].add(parent)

## Dependencies

In [9]:
to_inspect = ['IRN']
dependencies_formulas = []
dependencies_constants = []
dependencies_inputs = []
dependencies_unknown = []

while to_inspect:
    node = to_inspect.pop()
    
    if node in dependencies_formulas:
        continue
     
    for child in children_dict[node]:
        if child in formulas:
            if (child not in dependencies_formulas) and (child not in to_inspect):
                to_inspect.append(child)
        elif child in constants:
            if child not in dependencies_constants:
                dependencies_constants.append(child)
        elif child in inputs_list:
            if child not in dependencies_inputs:
                dependencies_inputs.append(child)
        elif child in unknown_variables:
            if child not in dependencies_unknown:
                dependencies_unknown.append(child)
        else:
            raise Exception('Unknown variable category : %s for parent %s.'%(child, node))

    dependencies_formulas.append(node)


In [10]:
len(dependencies_formulas)

4471

In [11]:
len(dependencies_constants)

215

In [12]:
len(dependencies_inputs)

1441

In [13]:
len(dependencies_unknown)

779

## Ignore useless variables

In [14]:
formulas_light = {k: formulas[k] for k in dependencies_formulas}
constants_light = {k: constants[k] for k in dependencies_constants}
inputs_light = dependencies_inputs
unknowns_light = dependencies_unknown

## Find cycles

### Adjacency matrix : too long

In [ ]:
index = {}
reverse_index = []
for i, k in enumerate(formulas_light):
    index[k] = i
    reverse_index.append(k)

In [ ]:
n = len(index)
adj = np.zeros((n, n))

In [ ]:
for parent in formulas_light:
    children = children_dict[parent]
    for child in children:
        if child in children_dict:
            adj[index[parent], index[child]] = 1

In [ ]:
current_power = 1
while current_power <= n:
    print(current_power)
    adj = np.dot(adj, adj)
    current_power += 1

In [ ]:
np.diag(adj).sum()

### Graph traversal : too long

In [25]:
children_light = {}
for formula in formulas_light:
    children_light[formula] = []
    for child in children_dict[formula]:
        if child in formulas_light:
            children_light[formula].append(child)

In [ ]:
def cycle_research(node, genealogy):
    if node in genealogy:
        print('Cycle detected : %s'%str(genealogy))
    else:
        for child in children_light[node]:
            child_genealogy = list(genealogy)
            child_genealogy.append(node)
            cycle_research(child, child_genealogy)
        

In [ ]:
cycle_research('IRN', [])

### Tarjan's strongly connected components algorithm

In [34]:
class State():
    def __init__(self):
        self.current_index = 0
        self.stack = []
        self.strongly_connected_components = []
        self.tarjan_indexes = {}
        self.low_link = {}
s = State()

In [35]:
def strong_connect(node, s):
    s.tarjan_indexes[node] = s.current_index
    s.low_link[node] = s.current_index
    s.current_index += 1
    s.stack.append(node)

    for child in children_light[node]:
        if child not in s.tarjan_indexes:
            strong_connect(child, s)
            s.low_link[node] = min(s.low_link[node], s.low_link[child])
        elif child in s.stack:
            s.low_link[node] = min(s.low_link[node], s.tarjan_indexes[child])

    if s.low_link[node] == s.tarjan_indexes[node]:
        new_component = []
        while s.stack[-1] != node:
            new_component.append(s.stack.pop())
        new_component.append(s.stack.pop())
        s.strongly_connected_components.append(new_component)

In [36]:
for node in formulas_light:
    if node not in s.tarjan_indexes:
        strong_connect(node, s)

In [42]:
component_size = [len(comp) for comp in s.strongly_connected_components]
c = collections.Counter(component_size)
c

Counter({1: 4471})

## Find good computing order

In [ ]:
# doesn't work : too much recursion
def find_order(node):
    if node not in simple_dependencies_dict:
        return
        
    deps = simple_dependencies_dict[node]
    is_leaf = True
    for dep in deps:
        if dep not in computing_order:
            find_order(dep)
            
    computing_order.append(node)
            

In [ ]:
root = 'IINET'
computing_order = []

to_inspect = [root]
while to_inspect:
    node = to_inspect[-1]
    is_leaf = True

    if node in simple_dependencies_dict:
        deps = simple_dependencies_dict[node]
    
        for dep in deps:
            if dep not in computing_order:
                is_leaf = False
                if dep not in to_inspect:
                    to_inspect.append(dep)
            
    if is_leaf:
        computing_order.append(node)
        to_inspect.pop()



In [ ]:
len(to_inspect)

In [ ]:
find_order(root)

In [ ]:
computing_order

In [ ]:
simple_dependencies_dict[root]

## Compute

In [ ]:
def product(l):
    accu = 1.
    for e in l:
        accu *= e
    return accu

def boolean_or(l):
    for e in l:
        if l:
            return 1.
    return 0.

def boolean_et(l):
    for e in l:
        if not l:
            return 0.
    return 1.

functions_mapping = {
    '+': sum,
    '*': product,
    '-': (lambda x: -x[0]),
    'positif': (lambda x: float(x[0]>0)),
    'positif_ou_nul': (lambda x: float(x[0]>=0)),
    'null': (lambda x: float(x[0]==0)),
    'operator:>=': (lambda x: float(x[0]>=x[1])),
    'operator:=': (lambda x: float(x[0]==x[1])),
    'ternary': (lambda x: x[1] if x[0] else x[2]),
    '/': (lambda x: x[0]/x[1] if x[1] else 0.),
    'max': max,
    'min': min,
    'inf': (lambda x: math.floor(x[0])),
    'arr': (lambda x: round(x[0])),
    'abs': (lambda x: abs(x[0])),
    'present': (lambda x: x[0] != 0.),
    'boolean:ou': boolean_or,
    'boolean:et': boolean_et,
    'dans': (lambda x: 1. if (x[0] in x[1:]) else 0.)
}


In [ ]:
def get_value(name):
    if name not in known_values:
        if name not in formulas_dict:
            print('Variable %s not found !'%name)
            known_values[name] = 0.
        else:
            formula = formulas_dict[name]
            known_values[name] = compute(formula)
            print('%d values known'%len(known_values))
    return known_values[name]

In [ ]:
def compute(node):
    nodetype = node['nodetype']

    if nodetype == 'symbol':
        name = node['name']
        value = get_value(name)
        return value 
        
    elif nodetype == 'float':
        value = node['value']
        return value
    
    elif nodetype == 'call':
        name = node['name']
        args = [compute(child) for child in node['args']]
        function = functions_mapping[name]
        try:
            value = function(args)
        except TypeError:
            print(args)
            print(function)
            raise TypeError()
        return value
        
    raise ValueError('Unknown type : %s'%nodetype)

In [ ]:
saisie = {'TSHALLOV': 30000.}
# http://www3.finances.gouv.fr/calcul_impot/2015/index.htm

known_values = {}

for c in constants:
    name = c['name']
    value = c['value']
    known_values[name] = value

for var in input_variables:
    name = var['name']
    alias = var['alias']
    
    known_alias = (alias in saisie)
    known_name = (name in saisie)
    
    if known_alias and known_name:
        raise ValueError('Variable defined twice.')
    if known_alias:
        value = saisie[alias]
        known_values[alias] = value
    else:
        known_values[alias] = 0.
        
    if known_name:
        value = saisie[name]
        known_values[name] = value
    else:
        known_values[name] = 0.
